## Description: 
### Sentiment analysis of Queen's songs

#### Libraries

In [1]:
! pip install lyrics_extractor

In [2]:
import os

import json
import requests
import pyjsonviewer

from functools import reduce
import operator

import pandas as pd
import numpy as np
import re

import lyricsgenius
from lyrics_extractor import SongLyrics

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk 
#nltk.download('punkt')
#stop_words = set(stopwords.words('english'))

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay


from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

#### Spotify Token
1. Get Auth for App
- Client_ID
- Client_Secret


2. Get Auth for queries
- token

In [4]:
def save_token (CLIENT_ID, CLIENT_SECRET):
    command = f"""echo "CLIENT_ID='{CLIENT_ID}'\nCLIENT_SECRET='{CLIENT_SECRET}'" > .env"""
    os.system(command)

In [5]:
def spotify_token ():
    
    """This function refreshes a 'queries token' for a given app on Spotify.
    queries token are refreshed every 3600 seconds
    returns: token as a string
    """

    # 1. Defining: credentials fot the app
    
    CLIENT_ID = os.getenv("CLIENT_ID")
    CLIENT_SECRET = os.getenv("CLIENT_SECRET")
    
    #2. Request
    
    body_params = {"grant_type":"client_credentials"}
    url = "https://accounts.spotify.com/api/token"
    response = requests.post(url, data=body_params, auth=(CLIENT_ID,CLIENT_SECRET))
    
    try:
        token = response.json()["access_token"]
        return token

    except:
        print("The request did not go through. Maybe check credentials?")

In [6]:
token = spotify_token()
token

'BQCS4sWBEUxnuhjoE-wNDCMy1h7h99pSXwLgOqcwVebQC-io_ypQoN5VHmNnSLpmdTRD7pUAu1f2XNSHD4GMM1L3iQOhYkFLYSbO-KvqxO9AGFHQifcK'

In [7]:
#token
headers = {"Authorization":f"Bearer {token}", 
            
}

#### Get Queen's (Studio) Albumns

1. url_base: core api url. 

2. [get several albumns query from api](https://developer.spotify.com/console/get-several-albums/)

3. split & join albumns ids for the query

In [8]:
url_base = 'https://api.spotify.com/v1/albums/'

In [9]:
query = 'https://api.spotify.com/v1/albums?ids=2hZTdsWsAqlgrYG6Nd6Egg%2C48MhNEYxMJvcBbqz85UTQP%2C5SBHID8qGG3x52zgoh2ilz%2C1GbtB4zTqAsyfZEsm1RZfx%2C4KZGe18wZJbXL6JLW4KyLc%2C7tB40pGzj6Tg0HePj2jWZt%2C5kffKW0sSLo6tkLg1veUGC%2C0pEfDPZko6TnNOgrZMe5nn%2C391ScNR3xKywWSpfDwP3n0%2C3h6SV9wHJtNL1YswZUJs8V%2C5RS9xkMuDmeVISqGDBmnSa%2C6reTSIf5MoBco62rk8T7Q1%2C4r9kNse038AoF2KfbqPUOS%2C58alCatewkjNm9IM1Ucj67%2C2yuTRGIackbcReLUXOYBqU'

In [10]:
# get queens albums
#ps: added first album 2 times so when split by '2%c' the result is added to the list. 
#queen_albums = albums_link.split('%2C')[1:]
#queen_albums

#albums_ids = ",".join([str(id) for id in queen_albums])
#albums_ids



In [11]:
#concat url base and albumns ID to make the query. Orignal URL query is only returning one album
#query = url_base + '?ids=' + albums_ids
#query

In [12]:
response = requests.get(query, headers=headers, verify=False).json()



c:\Users\danid\miniconda3\envs\ironhack\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.spotify.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [13]:
def get_albums (link):
    token = spotify_token()
    headers = {"Authorization":f"Bearer {token}"}
    
    query = 'https://api.spotify.com/v1/albums?ids=2hZTdsWsAqlgrYG6Nd6Egg%2C48MhNEYxMJvcBbqz85UTQP%2C5SBHID8qGG3x52zgoh2ilz%2C1GbtB4zTqAsyfZEsm1RZfx%2C4KZGe18wZJbXL6JLW4KyLc%2C7tB40pGzj6Tg0HePj2jWZt%2C5kffKW0sSLo6tkLg1veUGC%2C0pEfDPZko6TnNOgrZMe5nn%2C391ScNR3xKywWSpfDwP3n0%2C3h6SV9wHJtNL1YswZUJs8V%2C5RS9xkMuDmeVISqGDBmnSa%2C6reTSIf5MoBco62rk8T7Q1%2C4r9kNse038AoF2KfbqPUOS%2C58alCatewkjNm9IM1Ucj67%2C2yuTRGIackbcReLUXOYBqU'
    #query = 'https://api.spotify.com/v1/albums?ids=2hZTdsWsAqlgrYG6Nd6Egg%2C48MhNEYxMJvcBbqz85UTQP%2C5SBHID8qGG3x52zgoh2ilz%2C1GbtB4zTqAsyfZEsm1RZfx%2C4KZGe18wZJbXL6JLW4KyLc%2C7tB40pGzj6Tg0HePj2jWZt'
        
    try:    
        return requests.get(query, headers=headers).json()
    except:
        token = spotify_token()
        return requests.get(query, headers=headers).json()

In [14]:
len(response['albums'])

15

#### Filtered dictionary of Albums
1. Find relevant values for analysis (e.g. album_name, release_data, album_cover, etc)
2. Create function to loops through response and return all value in new, filtered, dict_

In [15]:
artist_name = response['albums'][5]['artists'][0]['name']

album_name = response['albums'][5]['name']
album_link = response['albums'][5]['external_urls']['spotify']
album_cover = response['albums'][5]['images'][0]['url']
album_popularity = response['albums'][1]['popularity']

release_date = response['albums'][5]['release_date']
total_tracks = response['albums'][5]['total_tracks']

track_name = response['albums'][5]['tracks']['items'][0]['name']
track_link = response['albums'][5]['tracks']['items'][0]['external_urls']['spotify']


print(f'artist_name: {artist_name}, album_name: {album_name}, album_link: {album_link}, album_cover: {album_cover}')
print(f'album_popularity: {album_popularity}, release_date: {release_date}, total_tracks: {total_tracks}')


artist_name: Queen, album_name: News Of The World (2011 Remaster), album_link: https://open.spotify.com/album/7tB40pGzj6Tg0HePj2jWZt, album_cover: https://i.scdn.co/image/ab67616d0000b2731f7077ae1018b5fbab08dfa8
album_popularity: 54, release_date: 1977-10-28, total_tracks: 11


In [16]:
def filtered_dict(response):

    filtered_data = []

    for album in response['albums']:
        dict_ = {}

        dict_['artist_name'] = album['artists'][0]['name']
        dict_['album_name'] = album['name']  
        dict_['album_link'] = album['external_urls']['spotify']
        dict_['album_cover'] = album['images'][0]['url']
        dict_['album_popularity'] = album['popularity']
        dict_['release_date'] = album['release_date']
        dict_['total_tracks'] = album['total_tracks']
        for i, track in enumerate(album['tracks']['items']):
            dict_[f'track_name_{i+1}'] = track['name']
            dict_[f'track_link{i+1}'] = track['external_urls']['spotify']
        filtered_data.append(dict_)
    return filtered_data 

In [17]:
queen = filtered_dict(response)
queen

[{'artist_name': 'Queen',
  'album_name': 'Queen (Deluxe Edition 2011 Remaster)',
  'album_link': 'https://open.spotify.com/album/2hZTdsWsAqlgrYG6Nd6Egg',
  'album_cover': 'https://i.scdn.co/image/ab67616d0000b2731de3187680d052813e7fd4d0',
  'album_popularity': 47,
  'release_date': '1973-07-13',
  'total_tracks': 16,
  'track_name_1': 'Keep Yourself Alive - 2011 Mix',
  'track_link1': 'https://open.spotify.com/track/3N25RDKNufh7Sz8v7fwtUP',
  'track_name_2': 'Doing Alright - Remastered 2011',
  'track_link2': 'https://open.spotify.com/track/72blnwOzPWQ3sQAD7Niw2X',
  'track_name_3': 'Great King Rat - Remastered 2011',
  'track_link3': 'https://open.spotify.com/track/2Wq1g93dCdtoK0rDfKVrDa',
  'track_name_4': 'My Fairy King - Remastered 2011',
  'track_link4': 'https://open.spotify.com/track/4dzTENQYOKn1BH3kFDC3kJ',
  'track_name_5': 'Liar - Remastered 2011',
  'track_link5': 'https://open.spotify.com/track/6hPAoJvuIsgPlxnoZnjPRk',
  'track_name_6': 'The Night Comes Down - Remastered 2

In [18]:
def filtered_dict_2(response):

    filtered_data = []

    for album in response['albums']:
        dict_ = {}

        dict_['artist_name'] = album['artists'][0]['name']
        dict_['album_name'] = album['name']  
        dict_['album_link'] = album['external_urls']['spotify']
        dict_['album_cover'] = album['images'][0]['url']
        dict_['album_popularity'] = album['popularity']
        dict_['release_date'] = album['release_date']
        dict_['total_tracks'] = album['total_tracks']
        track_names = []
        for track in album['tracks']['items']:
            #track_info = {}
            track_names.append(track['name'])
            dict_['tracks'] = track_names
        filtered_data.append(dict_)
    return filtered_data


In [19]:
queen_2 = filtered_dict_2(response)

#### Create data frame of filtered_dict 


In [20]:
df = pd.DataFrame(queen_2)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   artist_name       15 non-null     object
 1   album_name        15 non-null     object
 2   album_link        15 non-null     object
 3   album_cover       15 non-null     object
 4   album_popularity  15 non-null     int64 
 5   release_date      15 non-null     object
 6   total_tracks      15 non-null     int64 
 7   tracks            15 non-null     object
dtypes: int64(2), object(6)
memory usage: 1.1+ KB


In [21]:
df = df.explode('tracks')

In [22]:
df

,artist_name,album_name,album_link,album_cover,album_popularity,release_date,total_tracks,tracks
0,Queen,Queen (Deluxe Edition 2011 Remaster),https://open.spotify.com/album/2hZTdsWsAqlgrYG...,https://i.scdn.co/image/ab67616d0000b2731de318...,47,1973-07-13,16,Keep Yourself Alive - 2011 Mix
0,Queen,Queen (Deluxe Edition 2011 Remaster),https://open.spotify.com/album/2hZTdsWsAqlgrYG...,https://i.scdn.co/image/ab67616d0000b2731de318...,47,1973-07-13,16,Doing Alright - Remastered 2011
0,Queen,Queen (Deluxe Edition 2011 Remaster),https://open.spotify.com/album/2hZTdsWsAqlgrYG...,https://i.scdn.co/image/ab67616d0000b2731de318...,47,1973-07-13,16,Great King Rat - Remastered 2011
0,Queen,Queen (Deluxe Edition 2011 Remaster),https://open.spotify.com/album/2hZTdsWsAqlgrYG...,https://i.scdn.co/image/ab67616d0000b2731de318...,47,1973-07-13,16,My Fairy King - Remastered 2011
0,Queen,Queen (Deluxe Edition 2011 Remaster),https://open.spotify.com/album/2hZTdsWsAqlgrYG...,https://i.scdn.co/image/ab67616d0000b2731de318...,47,1973-07-13,16,Liar - Remastered 2011
...,...,...,...,...,...,...,...,...
14,Queen,Jazz (2011 Remaster),https://open.spotify.com/album/2yuTRGIackbcReL...,https://i.scdn.co/image/ab67616d0000b2737c39dd...,75,1978-11-10,13,Dreamer's Ball - Remastered 2011
14,Queen,Jazz (2011 Remaster),https://open.spotify.com/album/2yuTRGIackbcReL...,https://i.scdn.co/image/ab67616d0000b2737c39dd...,75,1978-11-10,13,Fun It - Remastered 2011
14,Queen,Jazz (2011 Remaster),https://open.spotify.com/album/2yuTRGIackbcReL...,https://i.scdn.co/image/ab67616d0000b2737c39dd...,75,1978-11-10,13,Leaving Home Ain't Easy - Remastered 2011
14,Queen,Jazz (2011 Remaster),https://open.spotify.com/album/2yuTRGIackbcReL...,https://i.scdn.co/image/ab67616d0000b2737c39dd...,75,1978-11-10,13,Don't Stop Me Now - Remastered 2011


#### Find the most popular albums

`Popularity:` Spotify's Popularity index is a 0-100 scale rating of how popular you are compared to every other artist on the platform. Some music marketing chatter suggests a PI of 20+ in the first few weeks will get you onto Release Radar and a PI of 30+ will get you onto Discover Weekly.

In [23]:
def group_by_popularity (df):
    return df.groupby("album_name").agg({"album_popularity": "mean"}).reset_index().sort_values(by="album_popularity", ascending=False)

In [24]:
group_by_popularity(df)

,album_name,album_popularity
12,The Game (2011 Remaster),77.0
6,Jazz (2011 Remaster),75.0
2,A Night At The Opera (2011 Remaster),74.0
8,News Of The World (2011 Remaster),74.0
4,Hot Space (2011 Remaster),72.0
0,A Day At The Races (Deluxe Edition 2011 Remaster),70.0
14,The Works (2011 Remaster),69.0
1,A Kind Of Magic (2011 Remaster),67.0
5,Innuendo (2011 Remaster),64.0
7,Made In Heaven (2011 Remaster),61.0


#### Exporting Albums DataFrame to CSV

In [25]:
df.to_csv('data/queen-albums_spotify.csv', index=False)

In [26]:
df = pd.read_csv('data/queen-albums_spotify.csv')

In [27]:
df.sample(10)

,artist_name,album_name,album_link,album_cover,album_popularity,release_date,total_tracks,tracks
172,Queen,The Game (2011 Remaster),https://open.spotify.com/album/58alCatewkjNm9I...,https://i.scdn.co/image/ab67616d0000b273056e90...,77,1980-06-27,10,Dragon Attack - Remastered 2011
57,Queen,A Day At The Races (Deluxe Edition 2011 Remaster),https://open.spotify.com/album/4KZGe18wZJbXL6J...,https://i.scdn.co/image/ab67616d0000b27367bc29...,70,1976-12-10,15,Tie Your Mother Down - Remastered 2011
22,Queen,Queen II (Deluxe Edition 2011 Remaster),https://open.spotify.com/album/48MhNEYxMJvcBbq...,https://i.scdn.co/image/ab67616d0000b273d92f69...,54,1974-03-08,16,The Fairy Feller's Master-Stroke - Remastered ...
103,Queen,A Kind Of Magic (2011 Remaster),https://open.spotify.com/album/0pEfDPZko6TnNOg...,https://i.scdn.co/image/ab67616d0000b273e98d08...,67,1986-06-02,9,Princes Of The Universe - Remastered 2011
16,Queen,Queen II (Deluxe Edition 2011 Remaster),https://open.spotify.com/album/48MhNEYxMJvcBbq...,https://i.scdn.co/image/ab67616d0000b273d92f69...,54,1974-03-08,16,Procession - Remastered 2011
101,Queen,A Kind Of Magic (2011 Remaster),https://open.spotify.com/album/0pEfDPZko6TnNOg...,https://i.scdn.co/image/ab67616d0000b273e98d08...,67,1986-06-02,9,Gimme The Prize - Remastered 2011
156,Queen,Flash Gordon (Deluxe Edition 2011 Remaster),https://open.spotify.com/album/4r9kNse038AoF2K...,https://i.scdn.co/image/ab67616d0000b273eea709...,44,1980-12-05,24,Escape From The Swamp - Remastered 2011
36,Queen,Sheer Heart Attack (2011 Remaster),https://open.spotify.com/album/5SBHID8qGG3x52z...,https://i.scdn.co/image/ab67616d0000b27322fcc5...,47,1974-11-08,13,Lily Of The Valley - Remastered 2011
9,Queen,Queen (Deluxe Edition 2011 Remaster),https://open.spotify.com/album/2hZTdsWsAqlgrYG...,https://i.scdn.co/image/ab67616d0000b2731de318...,47,1973-07-13,16,Seven Seas Of Rhye - Remastered 2011
27,Queen,Queen II (Deluxe Edition 2011 Remaster),https://open.spotify.com/album/48MhNEYxMJvcBbq...,https://i.scdn.co/image/ab67616d0000b273d92f69...,54,1974-03-08,16,"See What A Fool I've Been - Live BBC Session, ..."


#### Remove extra text from songs names so they will be easier to find: 
 

In [28]:
def clean_names(df):
    df['tracks'] = df['tracks'].apply(lambda x: re.sub(r'[-/].*$', '', x))
    df['album_name'] = df['album_name'].apply(lambda x: re.sub(r'\s+\(.*\)', '', x))
    return df

In [29]:
clean_names(df)



,artist_name,album_name,album_link,album_cover,album_popularity,release_date,total_tracks,tracks
0,Queen,Queen,https://open.spotify.com/album/2hZTdsWsAqlgrYG...,https://i.scdn.co/image/ab67616d0000b2731de318...,47,1973-07-13,16,Keep Yourself Alive
1,Queen,Queen,https://open.spotify.com/album/2hZTdsWsAqlgrYG...,https://i.scdn.co/image/ab67616d0000b2731de318...,47,1973-07-13,16,Doing Alright
2,Queen,Queen,https://open.spotify.com/album/2hZTdsWsAqlgrYG...,https://i.scdn.co/image/ab67616d0000b2731de318...,47,1973-07-13,16,Great King Rat
3,Queen,Queen,https://open.spotify.com/album/2hZTdsWsAqlgrYG...,https://i.scdn.co/image/ab67616d0000b2731de318...,47,1973-07-13,16,My Fairy King
4,Queen,Queen,https://open.spotify.com/album/2hZTdsWsAqlgrYG...,https://i.scdn.co/image/ab67616d0000b2731de318...,47,1973-07-13,16,Liar
...,...,...,...,...,...,...,...,...
189,Queen,Jazz,https://open.spotify.com/album/2yuTRGIackbcReL...,https://i.scdn.co/image/ab67616d0000b2737c39dd...,75,1978-11-10,13,Dreamer's Ball
190,Queen,Jazz,https://open.spotify.com/album/2yuTRGIackbcReL...,https://i.scdn.co/image/ab67616d0000b2737c39dd...,75,1978-11-10,13,Fun It
191,Queen,Jazz,https://open.spotify.com/album/2yuTRGIackbcReL...,https://i.scdn.co/image/ab67616d0000b2737c39dd...,75,1978-11-10,13,Leaving Home Ain't Easy
192,Queen,Jazz,https://open.spotify.com/album/2yuTRGIackbcReL...,https://i.scdn.co/image/ab67616d0000b2737c39dd...,75,1978-11-10,13,Don't Stop Me Now


In [30]:
df.sample(5)

,artist_name,album_name,album_link,album_cover,album_popularity,release_date,total_tracks,tracks
150,Queen,Flash Gordon,https://open.spotify.com/album/4r9kNse038AoF2K...,https://i.scdn.co/image/ab67616d0000b273eea709...,44,1980-12-05,24,The Ring (Hypnotic Seduction Of Dale)
170,Queen,Flash Gordon,https://open.spotify.com/album/4r9kNse038AoF2K...,https://i.scdn.co/image/ab67616d0000b273eea709...,44,1980-12-05,24,The Hero
37,Queen,Sheer Heart Attack,https://open.spotify.com/album/5SBHID8qGG3x52z...,https://i.scdn.co/image/ab67616d0000b27322fcc5...,47,1974-11-08,13,Now I'm Here
158,Queen,Flash Gordon,https://open.spotify.com/album/4r9kNse038AoF2K...,https://i.scdn.co/image/ab67616d0000b273eea709...,44,1980-12-05,24,Vultan's Theme (Attack Of The Hawk Men)
50,Queen,A Night At The Opera,https://open.spotify.com/album/1GbtB4zTqAsyfZE...,https://i.scdn.co/image/ab67616d0000b273e319ba...,74,1975-11-21,12,Sweet Lady


### TIME TO GET THE SONG LYRICS! 

1. [Using genius API](https://genius.com/api-clients) 

In [39]:
from getpass import getpass
import lyricsgenius


In [40]:
gen = getpass()

In [41]:
genius = lyricsgenius.Genius(gen)


In [43]:
# 1. starting small. Look for one queen song

queen = genius.search_artist("Queen", max_songs=1) 
cancion = "bohemian rhapsody"
song = queen.song(cancion)
song.lyrics

Searching for songs by Queen...

Song 1: "Bohemian Rhapsody"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "bohemian rhapsody" by Queen...
Done.


"TranslationsDeutschTürkçeไทย / Phasa ThaiEspañolPortuguêsفارسیBohemian Rhapsody Lyrics[Intro]\nIs this the real life? Is this just fantasy?\nCaught in a landslide, no escape from reality\nOpen your eyes, look up to the skies and see\nI'm just a poor boy, I need no sympathy\nBecause I'm easy come, easy go, little high, little low\nAny way the wind blows doesn't really matter to me, to me\n\n[Verse 1]\nMama, just killed a man\nPut a gun against his head, pulled my trigger, now he's dead\nMama, life had just begun\nBut now I've gone and thrown it all away\nMama, ooh, didn't mean to make you cry\nIf I'm not back again this time tomorrow\nCarry on, carry on as if nothing really matters\n\n[Verse 2]\nToo late, my time has come\nSends shivers down my spine, body's aching all the time\nGoodbye, everybody, I've got to go\nGotta leave you all behind and face the truth\nMama, ooh (Any way the wind blows)\nI don't wanna die\nI sometimes wish I'd never been born at all\nYou might also like[Guitar 

In [44]:
# 2. Create a function to get all lyrics from the df, taking into account that some might not be found. 
def get_lyrics (artist, song):
    try:
        artist_ = genius.search_artist(artist, max_songs=1)
        song_ = artist_.song(song)
        return song_.lyrics
    
    except:
        return np.nan

In [45]:
df["lyrics"] = df.apply(lambda row: get_lyrics (row["artist_name"], row["tracks"]), axis=1)

Searching for songs by Queen...

Song 1: "Bohemian Rhapsody"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Keep Yourself Alive " by Queen...
Done.
Searching for songs by Queen...

Song 1: "Bohemian Rhapsody"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Doing Alright " by Queen...
Done.
Searching for songs by Queen...

Song 1: "Bohemian Rhapsody"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Great King Rat " by Queen...
Done.
Searching for songs by Queen...

Searching for songs by Queen...

Searching for songs by Queen...

Song 1: "Bohemian Rhapsody"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "The Night Comes Down " by Queen...
Done.
Searching for songs by Queen...

Searching for songs by Queen...

Searching for songs by Queen...

Song 1: "Bohemian Rhapsody"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Jesus " by Queen...
Done.
Searchi

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   artist_name       194 non-null    object
 1   album_name        194 non-null    object
 2   album_link        194 non-null    object
 3   album_cover       194 non-null    object
 4   album_popularity  194 non-null    int64 
 5   release_date      194 non-null    object
 6   total_tracks      194 non-null    int64 
 7   tracks            194 non-null    object
 8   lyrics            138 non-null    object
dtypes: int64(2), object(7)
memory usage: 13.8+ KB


### Export CSV with song lyrics

In [49]:
df.to_csv("data/queen-lyrics.csv", index=False)